In [ ]:
pip install torch


In [1]:
import pandas as pd
import nltk
import torch
import numpy as np
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import time

# Load the pre-trained BERT models and tokenizers
model_names = ["bert-base-uncased", "bert-base-cased", "bert-large-uncased", "bert-large-cased", "bert-base-multilingual-cased"]

models = [AutoModel.from_pretrained(model_name) for model_name in model_names]
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in model_names]

data = pd.read_csv('/Users/uzairpachhapure/csu/Assignments/sem_4/project_final/new_data/cleaned_dat.csv')

# Tokenize and clean the questions
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)

data['CleanedQuestion'] = data['Question'].apply(preprocess_text)

# Encode questions and user input using BERT
def encode_text(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling over tokens
    return embeddings

# Accept user input
user_input = input("Enter your question: ")
user_input = preprocess_text(user_input)

# Initialize variables to keep track of the best model and its score
best_models = []
best_similarity_scores = []

# Calculate cosine similarity scores for each model
cosine_similarity_scores = []

for i, model_name in enumerate(model_names):
    model = models[i]
    tokenizer = tokenizers[i]
    
    # Encode all questions and user input
    question_embeddings = data['CleanedQuestion'].apply(lambda text: encode_text(text, model, tokenizer))
    user_input_embedding = encode_text(user_input, model, tokenizer)
    
    # Convert question embeddings to a 2D NumPy array
    question_embeddings = np.stack(question_embeddings)
    
    # Calculate cosine similarity between user input and dataset questions
    similarity_scores = cosine_similarity([user_input_embedding], question_embeddings)
    
    # Store the cosine similarity scores for this model
    cosine_similarity_scores.append(similarity_scores)
    
# Find the top 3 most similar questions for each model
num_top_questions = 3

for i, model_name in enumerate(model_names):
    top_question_indices = np.argsort(cosine_similarity_scores[i])[0][-num_top_questions:][::-1]
    top_similarity_scores = [cosine_similarity_scores[i][0][idx] for idx in top_question_indices]
    
    best_models.append(model_name)
    best_similarity_scores.append(list(zip(data['Question'][top_question_indices], top_similarity_scores)))

# Print the top 3 most similar questions and their cosine similarity scores for each model
for i in range(len(best_models)):
    print(f"Model {i + 1}: {best_models[i]}")
    for j, (question, score) in enumerate(best_similarity_scores[i]):
        print(f"Top {j + 1} Question: {question}")
        print(f"Cosine Similarity Score: {score:.4f}")
        print("-" * 20)
    print("=" * 50)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/uzairpachhapure/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/uzairpachhapure/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Enter your question: what is lebesgue measure?


KeyboardInterrupt: 

In [28]:
x = pd.read_csv('/Users/uzairpachhapure/csu/Assignments/sem_4/project_final/new_data/cleaned_dat.csv')

In [46]:
x.shape

(1600808, 12)

In [53]:
data = x.sample(n =10000, random_state = 42)

In [54]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [55]:


# Tokenize and clean the questions
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/uzairpachhapure/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/uzairpachhapure/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)


In [57]:

data['CleanedQuestion'] = data['Question'].apply(preprocess_text)


In [59]:

# Accept user input
user_input = input("Enter your question: ")
user_input = preprocess_text(user_input)

# Vectorize the questions and user input
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['CleanedQuestion'])
user_input_vector = tfidf_vectorizer.transform([user_input])

# Calculate cosine similarity between user input and dataset questions
similarity_scores = cosine_similarity(user_input_vector, tfidf_matrix)

# Find the indices of the top 3 most similar questions
top_similar_indices = similarity_scores.argsort()[0][-3:][::-1]

# Retrieve the corresponding answers for the top 3 questions
top_similar_questions = [data.iloc[index]['Question'] for index in top_similar_indices]
top_similar_answers = [data.iloc[index]['Answer'] for index in top_similar_indices]

# Print the top 3 similar questions and corresponding answers
print("Top 3 similar questions:")
for i, question in enumerate(top_similar_questions):
    print(f"Question {i + 1}: {question}")
    print(f"Answer {i + 1}: {top_similar_answers[i]}")
    print()

Enter your question: hyperparameter
Top 3 similar questions:
Question 1: Kellogg' theorem: Let m\ge1,0&lt;\alpha&lt;1. If \Omega\in \mathbb{R}^n is of class C^{m,\alpha}, then \log k\in C^{m-1,\alpha}(\partial\Omega), where k(y)=k_{x_0}(y)=k(x_0,y) is the Poisson kernel for the domain \Omega and x_0\in\Omega is fixed.

I am looking for the proof of this theorem, since I can not find it in Kellogg's book:Foundations of Potential Theory. I will be grateful if someone can tell me where I can get a complete proof for it.

Answer 1: nan

Question 2: Let X \in [-n,n] \cap \mathbb Z be a sum of n iid Rademacher random variables and let X_1,\ldots,X_k be k \ge 1 iid copies of X, and define the product Z_k := X_1 X_2 \ldots X_k. I'm interested in the moment-generating function (MGF) of Z, i.e \eta_k(t) := \mathbb E[e^{-tZ_k}]. Note that we can write X = 2B - n, where B is a Binomial r.v with parameter n and 1/2. Thus, by marginalizing w.r.t X_1, we have

\begin{split}
\eta_k(t) &amp;= \mathbb E

In [61]:
import pandas as pd
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/Users/uzairpachhapure/csu/Assignments/sem_4/project_final/new_data/cleaned_dat.csv')

# Sample dataset (replace with your actual dataset)
questions = df["Question"].tolist()
answers = df["Answer"].tolist()

# Load the pre-trained GloVe model (replace 'glove-wiki-gigaword-300' with the desired model name)
glove_model = api.load('glove-wiki-gigaword-300')

# Function to calculate the average word vector for a sentence
def average_word_vectors(sentence, model, num_features):
    words = sentence.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in words:
        if word in model:
            num_words += 1
            feature_vector = np.add(feature_vector, model[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Create word vectors for questions
question_vectors = [average_word_vectors(question, glove_model, num_features=300) for question in questions]

# Get user input
user_input = input("Enter your question: ")
print("\n")

# Preprocess user input and transform into a word vector
user_vector = average_word_vectors(user_input, glove_model, num_features=300)

# Calculate cosine similarities between user input and dataset questions
similarities = [cosine_similarity([user_vector], [question_vector])[0][0] for question_vector in question_vectors]

# Get the indices of the top 3 most similar questions
top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:3]

i = 1
# Retrieve and print corresponding answers
for idx in top_indices:
    print("Similar Question:", i, " : ", questions[idx])
    i = i + 1
    print("Answer:", answers[idx])
    print("Cosine Similarity:", similarities[idx])
    print("\n\n\n")

[===-----------------------------------------------] 6.2% 23.4/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[======--------------------------------------------] 12.6% 47.4/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[========------------------------------------------] 16.2% 61.0/376.1MB downloaded

...

[============================================------] 88.7% 333.5/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.0% 372.5/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Enter your question: I'm struggling to grasp intuition behind arc function. I know that it is a inverse function to corresponding trigonometric function (\sin,\cos...). Why their graph looks like it does?


Similar Question: 1  :  I'm struggling to grasp intuition behind arc function. I know that it is a inverse function to corresponding trigonometric function (\sin,\cos...).
Why their graph looks like it does?

Answer: There is a general answer for that (see the picture below): if you know the graph of f, then the graph of f^{-1} is that of f reflected about the line y=x.


Cosine Similarity: 0.9999999




Similar Question: 2  :  I need to prove that the inverse function of a linear function is also linear? I suppose that a linear function is defined like this: f(x)=ax+b but how can I prove that its inverse is linear?

Answer: y=ax+b\implies x=\frac{y-b}a=\frac1ay-\frac ba

In the comments it was already explained why it has to be \;a\neq 0\; ...

Cosine Similarity: 0.94047093




Sim

In [64]:
pip install pandas gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=549dad208012f4bb954f02f983a8011def1b13016090f8d2bf5fcdfe6d8a24de
  Stored in directory: /Users/uzairpachhapure/Library/Caches/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3513 sha256=1da39c046e24eeb741f2cd66c990c605de75318e159460f60c22924e9789ac4b
  Stored in directory: /Users/uzairpachhapure/Library/Caches/pip/wheels/9d/ff/2

In [66]:
import pandas as pd
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity

# Sample dataset (replace with your actual dataset)
questions = df["Question"].tolist()
answers = df["Answer"].tolist()



# Function to calculate the average word vector for a sentence
def average_word_vectors(sentence, model, num_features):
    words = sentence.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in words:
        if word in model:
            num_words += 1
            feature_vector = np.add(feature_vector, model[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Create word vectors for questions
question_vectors = [average_word_vectors(question, glove_model, num_features=300) for question in questions]

# Get user input
user_input = input("Enter your math-related question: ")
print("\n")

# Preprocess user input and transform into a word vector
user_vector = average_word_vectors(user_input, glove_model, num_features=300)

# Calculate cosine similarities between user input and dataset questions
similarities = [cosine_similarity([user_vector], [question_vector])[0][0] for question_vector in question_vectors]

# Get the indices of the top 3 most similar questions
top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:3]

i = 1
# Retrieve and print corresponding answers
for idx in top_indices:
    print("Similar Question:", i, " : ", questions[idx])
    i = i + 1
    print("Answer:", answers[idx])
    print("Cosine Similarity:", similarities[idx])
    print("\n\n\n")

Enter your math-related question: formula to calculate mean




KeyboardInterrupt: 

In [68]:
import pandas as pd
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
import time

# Sample dataset (replace with your actual dataset)
questions = df["Question"].tolist()
answers = df["Answer"].tolist()

#

# Function to calculate the average word vector for a sentence
def average_word_vectors(sentence, model, num_features):
    words = sentence.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in words:
        if word in model:
            num_words += 1
            feature_vector = np.add(feature_vector, model[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Create word vectors for questions
question_vectors = [average_word_vectors(question, glove_model, num_features=300) for question in questions]

while True:
    # Get user input
    user_input = input("Enter your math-related question (type 'exit' to end): ")
    if user_input.lower() == 'exit':
        break

    print("\nProcessing...")

    # Start the timer
    start_time = time.time()

    # Preprocess user input and transform into a word vector
    user_vector = average_word_vectors(user_input, glove_model, num_features=300)

    # Calculate cosine similarities between user input and dataset questions
    similarities = [cosine_similarity([user_vector], [question_vector])[0][0] for question_vector in question_vectors]

    # Get the indices of the top 3 most similar questions
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:3]

    # Retrieve and print corresponding answers
    i = 1
    for idx in top_indices:
        print("Similar Question:", i, " : ", questions[idx])
        i = i + 1
        print("Answer:", answers[idx])
        print("Cosine Similarity:", similarities[idx])
        print("\n")

    # Calculate and print the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processing time: {elapsed_time:.2f} seconds\n")

Enter your math-related question (type 'exit' to end): formula to calculate mean

Processing...
Similar Question: 1  :  Why \sum_{k=1}^n \left(\frac56\right)^{k-1}\cdot \left(\frac16\right)=1-\left(\frac56\right)^n??
Which formula I need to use to calculate it?

Thank you!

Answer: \sum_{k=1}^n\left(\frac56\right)^{k-1}\frac16=\frac15\sum_{k=1}^n\left(\frac56\right)^k\stackrel{\text{geom. series}}=\frac15\frac{\frac56-\left(\frac56\right)^{n+1}}{1-\frac56}=

{}

=\frac65\left[\frac56-\left(\frac56\right)^{n+1}\right]=1-\left(\frac56\right)^n

Cosine Similarity: 0.9008409


Similar Question: 2  :  Show that following formula is true:


\sum_{i=0}^{[n/2]}(-1)^i (n-2i)^n{n \choose i}=2^{n-1}n!


Using formula calculate

\sum_{i=0}^n(2i-n)^p{p \choose i}


Answer: nan
Cosine Similarity: 0.867305


Similar Question: 3  :  f(t)=\frac{\sin^3(\pi t)}{(\pi t)^3}
I want to calculate the Fourier transform.
I can't calculate this integral:
\int_0^\infty\frac{\sin^3(\pi t)}{(\pi t)^3}\cos(ut)\,\mat

In [69]:
import pandas as pd
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
import time

# Sample dataset (replace with your actual dataset)
questions = df["Question"].tolist()
answers = df["Answer"].tolist()

# Create an inverted index for the "questions" column
inverted_index = {}
for idx, question in enumerate(questions):
    words = question.split()
    for word in words:
        word = word.lower()  # Convert to lowercase to make the search case-insensitive
        if word not in inverted_index:
            inverted_index[word] = []
        inverted_index[word].append(idx)

# Function to calculate the average word vector for a sentence
def average_word_vectors(sentence, model, num_features):
    words = sentence.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in words:
        if word in model:
            num_words += 1
            feature_vector = np.add(feature_vector, model[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Create word vectors for questions
question_vectors = [average_word_vectors(question, glove_model, num_features=300) for question in questions]

while True:
    # Get user input
    user_input = input("Enter your math-related question (type 'exit' to end): ")
    if user_input.lower() == 'exit':
        break

    print("\nProcessing...")

    # Start the timer
    start_time = time.time()

    # Preprocess user input and transform into a word vector
    user_vector = average_word_vectors(user_input, glove_model, num_features=300)

    # Calculate cosine similarities between user input and dataset questions
    similarities = []

    # Search for matching words in the inverted index
    words = user_input.lower().split()
    matching_question_indices = set()
    for word in words:
        if word in inverted_index:
            matching_question_indices.update(inverted_index[word])

    # Calculate similarities only for the matching questions
    for idx in matching_question_indices:
        question_vector = question_vectors[idx]
        similarity = cosine_similarity([user_vector], [question_vector])[0][0]
        similarities.append((idx, similarity))

    # Sort by similarity score and get the top 3 most similar questions
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_indices = [idx for idx, _ in sorted_similarities[:3]]

    # Retrieve and print corresponding answers
    i = 1
    for idx in top_indices:
        print("Similar Question:", i, " : ", questions[idx])
        i = i + 1
        print("Answer:", answers[idx])
        print("Cosine Similarity:", sorted_similarities[i - 1][1])
        print("\n")

    # Calculate and print the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processing time: {elapsed_time:.2f} seconds\n")

Enter your math-related question (type 'exit' to end): formula to calculate mean

Processing...
Similar Question: 1  :  Why \sum_{k=1}^n \left(\frac56\right)^{k-1}\cdot \left(\frac16\right)=1-\left(\frac56\right)^n??
Which formula I need to use to calculate it?

Thank you!

Answer: \sum_{k=1}^n\left(\frac56\right)^{k-1}\frac16=\frac15\sum_{k=1}^n\left(\frac56\right)^k\stackrel{\text{geom. series}}=\frac15\frac{\frac56-\left(\frac56\right)^{n+1}}{1-\frac56}=

{}

=\frac65\left[\frac56-\left(\frac56\right)^{n+1}\right]=1-\left(\frac56\right)^n

Cosine Similarity: 0.867305


Similar Question: 2  :  Show that following formula is true:


\sum_{i=0}^{[n/2]}(-1)^i (n-2i)^n{n \choose i}=2^{n-1}n!


Using formula calculate

\sum_{i=0}^n(2i-n)^p{p \choose i}


Answer: nan
Cosine Similarity: 0.859267


Similar Question: 3  :  f(t)=\frac{\sin^3(\pi t)}{(\pi t)^3}
I want to calculate the Fourier transform.
I can't calculate this integral:
\int_0^\infty\frac{\sin^3(\pi t)}{(\pi t)^3}\cos(ut)\,\math

KeyboardInterrupt: Interrupted by user

This is an example . How to remove it? Another random math expression ...
